## Tidy data (aka structuring your dataset to facilitate your analysis)
- Each variable forms a column and contains values
- Each observation forms a row
- Each type of observational unit forms a table

A few definitions:

- Variable: A measurement or an attribute. Height, weight, sex, etc.
- Value: The actual measurement or attribute. 152 cm, 80 kg, female, etc.
- Observation: All values measure on the same unit. Each person.

#### An example of a messy dataset:
    
|              | Treatment A | Treatment B |
|--------------|-------------|-------------|
| John Smith   | \-          | 2           |
| Jane Doe     | 16          | 11          |
| Mary Johnson | 3           | 1           |

#### An example of a tidy dataset:
| Name         | Treatment | Result |
|--------------|-----------|--------|
| John Smith   | a         | \-     |
| Jane Doe     | a         | 16     |
| Mary Johnson | a         | 3      |
| John Smith   | b         | 2      |
| Jane Doe     | b         | 11     |
| Mary Johnson | b         | 1      |


In [4]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

### What is a messy dataset?
- Column headers are values, not variable names.
- Multiple variables are stored in one column.
- Variables are stored in both rows and columns.
- Multiple types of observational units are stored in the same table.
- A single observational unit is stored in multiple tables.

In [5]:
df = pd.read_csv("./bcmb_bootcamp2020/day2/data/tb-raw.csv")
df

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


##### What is wrong with the above data set? Discuss in your groups.  

To fix this data set we will do a series of operations outlined below: 

- Use melt function to unpivot the dataframe from wide to long format <br/>
    - more information about this function here: https://pandas.pydata.org/docs/reference/api/pandas.melt.html

In [6]:
df = pd.melt(df, id_vars=["country","year"], value_name="num_cases", var_name="sex_and_age")
# we use the head function to visualize the top rows of the dataframe
df.head()

,country,year,sex_and_age,num_cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


- Now we will try to separate the column sex_and_age to four three different columns and save them in a temporary dataframe: 

    - See here for more information: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html

In [7]:
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})") 
tmp_df.columns = ["sex", "age_lower", "age_upper"]
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]
tmp_df.head()

,sex,age_lower,age_upper,age
0,m,0,14,0-14
1,m,0,14,0-14
2,m,0,14,0-14
3,m,0,14,0-14
4,m,0,14,0-14


- Now that we want to concatenate these two dataframes back together:

    - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [9]:
df = pd.concat([df, tmp_df], axis=1)
df.head()

,country,year,sex_and_age,num_cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14
2,AF,2000,m014,52.0,m,0,14,0-14
3,AG,2000,m014,0.0,m,0,14,0-14
4,AL,2000,m014,2.0,m,0,14,0-14


- Everything looks right, except we have some redundant columns. We would want to get rid of NaN values as well:

In [10]:
# in order to save time and space by storing new dataframes in a variable, we can use inplace=True:
df.drop(['sex_and_age',"age_lower","age_upper"], axis=1,inplace=True)
# remove Nan values:
df.dropna(inplace=True)
# sort the dataframe based on columns:
df.sort_values(["country", "year", "sex", "age"],ascending=True,inplace=True)

In [11]:
df

,country,year,num_cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54
...,...,...,...,...,...
38,AR,2000,402.0,m,35-44
48,AR,2000,419.0,m,45-54
58,AR,2000,368.0,m,55-64
49,AS,2000,1.0,m,45-54
